# **点云**


### **点云是什么**
点云是某个坐标系下的点的数据集。

点包含了丰富的信息，包括三维坐标X，Y，Z、颜色、分类值、强度值、时间等等，不一一列举。

在我看来点云可以将现实世界原子化，通过高精度的点云数据可以还原现实世界。

万物皆点云。

In [2]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

### **读取显示点云（Point Cloud）**

In [9]:
print("Load a ply point cloud, print it, and render it")
ply_point_cloud = o3d.data.PLYPointCloud() # 官方的一个数据
pcd = o3d.io.read_point_cloud(ply_point_cloud.path) # 读取点云数据
print(pcd)
print(np.asarray(pcd.points))
# 可视化点云
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024]) # 设置相机位置


Load a ply point cloud, print it, and render it
PointCloud with 196133 points.
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]


### **体素（Voxel）下采样**
点被分桶到体素中。

每个被占用的体素通过平均内部的所有点来生成一个点

In [10]:
downpcd = pcd.voxel_down_sample(voxel_size=0.05) # voxel_size为采样的半径
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

### **法线（Normal）估计**

In [11]:
downpcd.estimate_normals( # normals为法线的意思
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)) # 以半径0.1，最多30个点为一个面来生成法线
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True)

### 裁减（crop）点云

In [12]:
# 第一步：读取文件
demo_crop_data = o3d.data.DemoCropPointCloud()
pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)
# 第二步；裁剪指定位置的的点，为json文件，vol就是
vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path)
chair = vol.crop_point_cloud(pcd)
# 第三步；显示
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

### **绘制点云**

In [14]:
chair.paint_uniform_color([1, 0.706, 0])
# 更改点云的颜色
o3d.visualization.draw_geometries([chair],)

### **绘制边界体积**
get_axis_aligned_bounding_box() 是按照轴对其包围盒

get_oriented_bounding_box() 按照点云有方向的包围盒


In [15]:
aabb = chair.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
obb = chair.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([chair, aabb, obb],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

In [ ]:
# https://blog.csdn.net/qq_22822335/article/details/50930437

### **凸包（Convex Hull）**

In [16]:
# 兔子点云
bunny = o3d.data.BunnyMesh()
mesh = o3d.io.read_triangle_mesh(bunny.path)
mesh.compute_vertex_normals()

pcl = mesh.sample_points_poisson_disk(number_of_points=2000)
hull, _ = pcl.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([pcl, hull_ls])

In [ ]:
# 凸包的作用与原理 https://blog.csdn.net/HouszChina/article/details/79251474

In [ ]:
# https://blog.csdn.net/QLeelq/article/details/122136481 这篇文章涉及后续所有内容